In [ ]:
import torch
from transformers import AutoTokenizer
from utils.NERDataset import NERDataset
from utils.NERModel import NERModel
from utils.saver import tokenizer_loader

tokenizer = tokenizer_loader(AutoTokenizer, "bert-base-chinese")
train_dataset = NERDataset(tokenizer=tokenizer, filename="./data/msra.train")
dev_dataset = NERDataset(tokenizer=tokenizer, filename="./data/msra.dev")
test_dataset = NERDataset(tokenizer=tokenizer, filename="./data/msra.test")

assert train_dataset.id_label == dev_dataset.id_label
assert train_dataset.id_label == test_dataset.id_label

model = NERModel(train_dataset.num_labels, model_name="bert-base-chinese")

In [ ]:
from utils.metrics import calc_acc

def test(dataset, model):
    loader = torch.utils.data.DataLoader(dataset, batch_size=dataset.length)
    _, (X, Y) = next(enumerate(loader))
    predict, ans, loss = model(X, Y)
    print(calc_acc(predict, ans))

In [ ]:
import tqdm

LEARNING_RATE = 5e-3
EPOCH = 1
BATCH_SIZE = 4

def train(dataset, model):
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
    loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE)

    for epoch_index in range(EPOCH):
        for batch_index, (batch_X, batch_Y) in enumerate(loader):
            _, __, loss = model(batch_X, batch_Y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [ ]:
test(dataset, model)

In [ ]:
train(dataset, model)

In [ ]:
test(dataset, model)